In [1]:
%pip install --upgrade --quiet  duckduckgo-search
%pip install --upgrade --quiet  wikipedia
%pip install chromadb

%pip install InstructorEmbedding
%pip install sentence-transformers

%pip install --upgrade --quiet  gpt4all > /dev/null


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.utilities import GoogleSerperAPIWrapper


import os
import pprint

os.environ["SERPER_API_KEY"] = "e1f8cd9de27d3b78f9ab3428167883002e76c33b"


from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
dd_search = DuckDuckGoSearchRun()
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
google_search = GoogleSerperAPIWrapper()

In [4]:
dd = dd_search.run("How is milk Made?")
google = google_search.run("How is milk made?")
wiki = wikipedia.run("How is milk made?")
pprint.pp(wiki)


/home/naheed/Desktop/gpt4all/env/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:47: UserWarning: DDGS running in an async loop. This may cause errors. Use AsyncDDGS instead.
  with DDGS() as ddgs:


('Page: Fat content of milk\n'
 'Summary: The fat content of milk is the proportion of milk, by weight,:\u200a'
 '266\u200a\n'
 "made up by butterfat. The fat content, particularly of cow's milk, is "
 'modified to make a variety of products. The fat content of milk is usually '
 'stated on the container, and the color of the label or milk bottle top '
 'varied to enable quick recognition.\n'
 '\n'
 'Page: Skimmed milk\n'
 'Summary: Skimmed milk (British English), or skim milk (American English), is '
 'made when all the milkfat is removed from whole milk. It tends to contain '
 'around 0.1% fat.\n'
 '\n'
 'Page: Oat milk\n'
 'Summary: Oat milk is a plant milk derived from whole oat (Avena spp.) grains '
 'by extracting the plant material with water. Oat milk has a creamy texture '
 'and mild oatmeal-like flavor, and is manufactured in various flavors, such '
 'as sweetened, unsweetened, vanilla, and chocolate.\n'
 'Unlike other plant milks having origins as early as the 13th century, 

In [5]:

#splitting the text into

# print(len(data))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts1 = text_splitter.split_text(dd)
texts2 = text_splitter.split_text(google)
texts3 = text_splitter.split_text(wiki)

print(len(dd))
print(len(google))
print(len(wiki))

print(len(texts1))
print(len(texts2))
print(len(texts3))
# print(texts[0])

1392
275
1750
2
1
3


In [6]:
%pip install -Uq chromadb numpy datasets
# Import Chroma and instantiate a client. The default Chroma client is ephemeral, meaning it will not save to disk.
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(allow_reset=True))

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Create a new Chroma collection to store the supporting evidence. We don't need to specify an embedding fuction, and the default will be used.
client.reset()
collection = client.create_collection("searched_data_on_web")

In [8]:

start = 0
finish = len(texts1)

collection.add(
    ids=[str(i) for i in range(start, finish)],
    documents= texts1[:len(texts1)],
    metadatas=[{"source":"DuckDuckGoSearch"} for i in range(start,finish)] 
)


start = len(texts1)
finish = len(texts1)+len(texts2)
collection.add(
    ids=[str(i) for i in range(start, finish)],
    documents= texts2[:len(texts2)],
    metadatas=[{"source":"GoogleSearch"}for i in range(start,finish)] 
)


start = len(texts1)+len(texts2)
finish = len(texts1)+len(texts2)+len(texts3)
collection.add(
    ids=[str(i) for i in range(start, finish)],
    documents= texts3[:len(texts3)],
    metadatas=[{"source":"WikiSearch"}for i in range(start,finish)] 
)

In [9]:
results = collection.query(
    query_texts=["How is milk made?"],
    n_results=10
)

results

Number of requested results 10 is greater than number of elements in index 6, updating n_results = 6


{'ids': [['2', '0', '4', '3', '1', '5']],
 'distances': [[0.8157816529273987,
   0.9763622283935547,
   1.0069234371185303,
   1.0191516876220703,
   1.1000409126281738,
   1.1715294122695923]],
 'metadatas': [[{'source': 'GoogleSearch'},
   {'source': 'DuckDuckGoSearch'},
   {'source': 'WikiSearch'},
   {'source': 'WikiSearch'},
   {'source': 'DuckDuckGoSearch'},
   {'source': 'WikiSearch'}]],
 'embeddings': None,
 'documents': [["Cows are milked using vacuum cups which are attached to the cow's teats. The milk is sent through stainless steel pipes to large refrigerated vats, then stored at 5°C or less. Within 48 hours, milk is taken in tankers to a milk factory where it's pasteurised and homogenised.",
   "Nutrition for infant mammals Breastfeeding to provide a mother's milk A goat kid feeding on its mother's milk In almost all mammals, milk is fed to infants through breastfeeding, either directly or by expressing the milk to be stored and consumed later. The early milk from mammals 

In [10]:
results['documents']


for i in range(len(results['ids'][0])):
    print(f'data from id {i}')
    print(f'data from source {results["metadatas"][0][0]["source"]}')
    print(f'Answer : {results["documents"][0][i]}')
    print("------------------------------------------------")


data from id 0
data from source GoogleSearch
Answer : Cows are milked using vacuum cups which are attached to the cow's teats. The milk is sent through stainless steel pipes to large refrigerated vats, then stored at 5°C or less. Within 48 hours, milk is taken in tankers to a milk factory where it's pasteurised and homogenised.
------------------------------------------------
data from id 1
data from source GoogleSearch
Answer : Nutrition for infant mammals Breastfeeding to provide a mother's milk A goat kid feeding on its mother's milk In almost all mammals, milk is fed to infants through breastfeeding, either directly or by expressing the milk to be stored and consumed later. The early milk from mammals is called colostrum. Processing Raw vs. pasteurized Bottom line You may know cow's milk for its calcium content, but it has much more to offer. A single cup provides nearly every nutrient your body needs. This... 961. Dairy product, milk and any of the foods made from milk, including 

In [12]:
res = ''
for i in range(min(5 , len(results['ids'][0]))):
    res += results["documents"][0][i]
 
res

"Cows are milked using vacuum cups which are attached to the cow's teats. The milk is sent through stainless steel pipes to large refrigerated vats, then stored at 5°C or less. Within 48 hours, milk is taken in tankers to a milk factory where it's pasteurised and homogenised.Nutrition for infant mammals Breastfeeding to provide a mother's milk A goat kid feeding on its mother's milk In almost all mammals, milk is fed to infants through breastfeeding, either directly or by expressing the milk to be stored and consumed later. The early milk from mammals is called colostrum. Processing Raw vs. pasteurized Bottom line You may know cow's milk for its calcium content, but it has much more to offer. A single cup provides nearly every nutrient your body needs. This... 961. Dairy product, milk and any of the foods made from milk, including butter, cheese, ice cream, yogurt, and condensed and dried milk. Cow's milk is by far the principal type of milk used in dairy products worldwide. Learn more